In [1]:
#james chartouni
import lightgbm as lgb
import pandas as pd
import numpy as np
from numpy import inf
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler

In [2]:
training_data = pd.read_csv("cleaned_input/train_consolidated.csv")
training_data = training_data.drop(['msno'], axis=1)

In [3]:
#split data 
y = training_data["is_churn"].values
X = training_data.drop(["is_churn"], axis=1).values
#remove infinite values? 
X[X == -inf] = 0
X[X == inf] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, shuffle=True)#change to stratified shuffle split

In [9]:
#logistic regression 

logistic_pipe = make_pipeline(LogisticRegression())
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



In [6]:
#Light GBM 
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# generate a feature name
feature_name = X_train.dtype.names

print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,  # eval training data
              )

# save model to file
gbm.save_model('model.txt')

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))


Start training...
[1]	training's binary_logloss: 0.644421
[2]	training's binary_logloss: 0.600332
[3]	training's binary_logloss: 0.560244
[4]	training's binary_logloss: 0.523647
[5]	training's binary_logloss: 0.490118
[6]	training's binary_logloss: 0.459299
[7]	training's binary_logloss: 0.433141
[8]	training's binary_logloss: 0.408971
[9]	training's binary_logloss: 0.384348
[10]	training's binary_logloss: 0.361511
[11]	training's binary_logloss: 0.340291
[12]	training's binary_logloss: 0.320542
[13]	training's binary_logloss: 0.302131
[14]	training's binary_logloss: 0.284948
[15]	training's binary_logloss: 0.268888
[16]	training's binary_logloss: 0.25386
[17]	training's binary_logloss: 0.239787
[18]	training's binary_logloss: 0.226588
[19]	training's binary_logloss: 0.214203
[20]	training's binary_logloss: 0.202571
[21]	training's binary_logloss: 0.191637
[22]	training's binary_logloss: 0.181352
[23]	training's binary_logloss: 0.171672
[24]	training's binary_logloss: 0.162554
[25]	tra